In [1]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None
exp_path = Path.cwd().resolve() # experiment path
# visible GPUs
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cv2 as cv
import numpy as np
import omidb

# Mask
Let's try to make the mask of the breast first.

In [4]:
def get_normal_BBox(image):
    img = cv.threshold(image, 0, 255, cv.THRESH_BINARY)[1]
    #plot.imshow(img,'gray')
    #plot.show() # display binary image
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img2 = np.zeros(output.shape,dtype=np.uint8)
    img2[output == max_label] = 255
    contours, hierarchy = cv.findContours(img2,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    cnt = contours[0]
    aux_im = img2
    x,y,w,h = cv.boundingRect(cnt)
    cv.rectangle(aux_im,(x,y),(x+w,y+h),(255,0,0),5)
    out_bbox = omidb.mark.BoundingBox(x, y, x+w, y+h)
    
    return out_bbox, img2 # returns bounding box and mask image.

In [ ]:
# we use the healthy images
images_directory = repo_path 